In [1]:
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
import tqdm

torch의 기본적인 라이브러리를 호출한다.

In [2]:
from torchvision.datasets import FashionMNIST
from torchvision import transforms

torchvision에 FashionMNIST 데이터가 아예 내장되어 있다.

In [3]:
from google.colab import drive
drive.mount('/content/gdrive/', force_remount = True)

Mounted at /content/gdrive/


In [4]:
%cd /content/gdrive/My Drive/pytorch/

/content/gdrive/My Drive/pytorch


In [5]:
train = FashionMNIST('/fashionmnist', train = True, download = True, transform = transforms.ToTensor())
test = FashionMNIST('/fashionmnist', train = False, download = True, transform = transforms.ToTensor())


Extracting /fashionmnist/FashionMNIST/raw/train-images-idx3-ubyte.gz to /fashionmnist/FashionMNIST/raw




Extracting /fashionmnist/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /fashionmnist/FashionMNIST/raw




Extracting /fashionmnist/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /fashionmnist/FashionMNIST/raw




Extracting /fashionmnist/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /fashionmnist/FashionMNIST/raw



/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [6]:
bs = 64

batch_size는 64로 설정

In [7]:
tr_loader = DataLoader(train, batch_size = bs, shuffle = True)
te_loader = DataLoader(test, batch_size = bs, shuffle = True)

pytorch의 DataLoader를 활용해 데이터를 담는다.

In [8]:
class Flatten_Layer(nn.Module) :
    def forward(self, x) :
        sizes = x.size()
    return x.view(sizes[0], -1)

CNN 출력값을 Linaer로 보내기 위해 만든 클래스

In [9]:
conv_net = nn.Sequential(
    nn.Conv2d(1, 32, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(32),
    nn.Dropout2d(.3),
    nn.Conv2d(32, 64, 5),
    nn.MaxPool2d(2),
    nn.ReLU(),
    nn.BatchNorm2d(64),
    nn.Dropout2d(.2),
    Flatten_Layer()
)

32 채널로 시작해서 64채널로 출력하여 마지막 Flatten_Layer에 input으로 넣는다.

In [ ]:
te_input = torch.ones(1, 1, 28, 28)
conv_output_size = conv_net(te_input).size()[-1]

CNN 레이어를 통과한 출력 값의 크기를 지정해서 이후 Linear input의 shape로 사용한다.

In [17]:
mlp = nn.Sequential(
    nn.Linear(conv_output_size, 128),
    nn.ReLU(),
    nn.BatchNorm1d(128),
    nn.Dropout(.25),
    nn.Linear(128, 10)
)

최종 출력 값 10개 클래스의 값을 반환하도록 한다.

In [18]:
net = nn.Sequential(conv_net, mlp)

conv_net, mlp를 Sequential로 연결한다.

In [26]:
def eval_net(net, data_loader, device = 'cuda:0') :

    net.eval() # 신경망을 평가 모드로 설정

    
    ys = []
    y_preds = []

    for x, y in data_loader :
        
        x = x.to(device) # 'cuda:0'으로 할당한다.
        y = y.to(device) # 'cuda:0'으로 할당한다.

        with torch.no_grad() :
          _, y_pred = net(x).max(1)
        ys.append(y)
        y_preds.append(y_pred)

    ys = torch.cat(ys)
    y_preds = torch.cat(y_preds)

    acc = (ys == y_preds).float().sum() / len(ys)

    return acc.item()

Cat 함수는 concatenate를 해주는 함수로 pd.concat과 동일하다고 생각하면 된다.

In [36]:
def train_net(net, train_loader, test_loader, optimizer = optim.Adam, loss_fn = nn.CrossEntropyLoss(), n_iter = 10, device = 'cuda:0') :

    tr_losses = []
    tr_acc = []
    val_acc = []
    optimizer = optimizer(net.parameters())

    for epoch in range(n_iter) :

        running_loss = 0
        net.train() # 신경망을 훈련 모드로 설정
        n = 0
        acc = 0

        for i, (xx, yy) in tqdm.tqdm(enumerate(train_loader), total = len(train_loader)) :
            xx = xx.to(device)
            yy = yy.to(device)
            h = net(xx)
            loss = loss_fn(h, yy)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            n += len(xx)
            _, y_pred = h.max(1)
            acc += (yy == y_pred).float().sum().item()
            
    tr_losses.append(running_loss / i)
    tr_acc.append(acc / n)

    val_acc.append(eval_net(net, test_loader, device))

    print(epoch, tr_losses[-1], tr_acc[-1], val_acc[-1], flush = True)

기존 pytorch process를 적용한다.

In [37]:
net.to('cuda:0')

Sequential(
  (0): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1))
    (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (2): ReLU()
    (3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Dropout2d(p=0.3, inplace=False)
    (5): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1))
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): ReLU()
    (8): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Dropout2d(p=0.2, inplace=False)
    (10): Flatten_Layer()
  )
  (1): Sequential(
    (0): Linear(in_features=1024, out_features=128, bias=True)
    (1): ReLU()
    (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.25, inplace=False)
    (4): Linear(in_features=128, out_features=10, bias=True)
  )
)

뭔가 keras의 summary와 동일한듯?

In [38]:
train_net(net, tr_loader, te_loader, n_iter = 20, device = 'cuda:0')

100%|██████████| 938/938 [00:08<00:00, 112.52it/s]


0 0.24573323034394132 0.9101333333333333 0.9070999622344971


100%|██████████| 938/938 [00:07<00:00, 119.83it/s]


1 0.23316186609237655 0.9143 0.9114999771118164


100%|██████████| 938/938 [00:07<00:00, 120.78it/s]


...

17 0.15882952823010427 0.93995 0.915399968624115


100%|██████████| 938/938 [00:07<00:00, 121.18it/s]


18 0.15755184891304694 0.94055 0.9230999946594238


100%|██████████| 938/938 [00:07<00:00, 120.69it/s]


19 0.15302411430856272 0.9426 0.9116999506950378


아직 잘 모르겠다. 토이 프로젝트로 더 해봐야 할 듯?